In [ ]:
import cv2
import numpy as np
from numba import njit
import matplotlib as plt
%matplotlib inline
alp = 10000000
beta = 1
x = 1000
y = 1000
N = 5

In [ ]:
Image = []
for i in range(1,N+1,1):
    Image.append(cv2.imread('data/image_0' + str(i) + '.png'))
Image = np.array(Image)

In [ ]:
Mask = []
for i in range(1,N+1,1):
    Mask.append(cv2.imread('data/mask_0' + str(i) + '.png',0))
Mask = np.array(Mask)

In [ ]:
def Create_Mask(mask):
    contours, hierarchy = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    len_contour = len(contours)
    contour_list = []
    for i in range(len_contour):
        drawing = np.zeros_like(mask, np.uint8)
        img_contour = cv2.drawContours(drawing, contours, i, (255, 255, 255), -1)
        contour_list.append(img_contour)

    out = sum(contour_list)
    return out

In [ ]:
Mask = np.array(list(map(Create_Mask, Mask)))

In [ ]:
@njit
def Create_B_q():
    B = np.zeros((N,x,y))
    q = np.zeros((N,x,y))
    for k in range(N):
        for i in range(x):
            for j in range(y):
                if Mask[k][i][j] == 0: 
                    B[k][i][j] = 0
                else: B[k][i][j] = 1
                q[k][i][j] = alp*(1-B[k][i][j])
    return B,q

B,q = Create_B_q()

In [ ]:
@njit()
def Create_g():
    g = np.zeros((N,N,x,y-1))
    for k_1 in range(N):
        for k_2 in range(N):
            for i in range(x):
                for j in range(y-1):
                    r1 = Image[k_1][i][j]-Image[k_2][i][j]
                    r2 = Image[k_1][i][j+1]-Image[k_2][i][j+1]
                    g[k_1,k_2,i,j] = beta*((r1[0]**2+r1[1]**2+r1[2]**2)**(0.5)+(r2[0]**2+r2[1]**2+r2[2]**2)**(0.5))
    return g

g = Create_g()

In [ ]:
@njit
def Create_f():
    f = np.zeros((N,x,y-1))
    for k in range(N):
        for i in range(x):
            for j in range(y-2,-1,-1):
                if j == y-2:
                    f[k][i][j] = min([q[m][i][j+1]+g[m][k][i][j] for m in range(N)])
                else:
                    f[k][i][j] = min([q[m][i][j+1]+g[m][k][i][j]+f[k][i][j+1] for m in range(N)])
    return f

f = Create_f()


In [ ]:
@njit
def Create_K_opt():
    K_opt = np.ones((x,y))
    for i in range(x):
        for j in range(y):
                if j == 0:
                    min = q[0][i][j]+f[0][i][j]
                    d = 0
                    for m in range(1,N,1):
                        cur = q[m][i][j]+f[m][i][j]
                        if cur < min:
                            min = cur
                            d = m
                    K_opt[i][j] = d
                if j == y-1:
                    min = q[0][i][j]+g[0][int(K_opt[i][j-1])][i][j]
                    d = 0
                    for m in range(1,N,1):
                        cur = q[m][i][j]+g[m][int(K_opt[i][j-1])][i][j]
                        if cur < min:
                            min = cur
                            d = m
                    K_opt[i][j] = d
                else:
                    min = q[0][i][j]+g[0][int(K_opt[i][j-1])][i][j]+f[0][i][j]
                    d = 0
                    for m in range(1,N,1):
                        cur = q[m][i][j]+g[m][int(K_opt[i][j-1])][i][j]+f[m][i][j]
                        if cur <= min:
                            min = cur
                            d = m
                    K_opt[i][j] = d
    return K_opt

K_opt = Create_K_opt()

In [ ]:
a = 0
s = 0
for i in range(N):
    a = np.sum(K_opt == i)
    s += a
    print(a)

print(s)

In [ ]:
Res_Im = cv2.imread('data/image_0' + str(1) + '.png')
for i in range(x):
    for j in range(y):
        for k in range(3):
            Res_Im[i][j][k] = Image[int(K_opt[i][j])][i][j][k]

In [ ]:
plt.pyplot.imshow(Res_Im)